In [ ]:
      "source": [
"# 4) Outlier Detection / Removal\n"
      ],
      "metadata": {
        "id": "SnB650VBMEdv"
      }
    },
    {
      "cell_type": "markdown",
      "source": [
        "### 4.1 Detection Techniques\n"
      ],
      "metadata": {
        "id": "kj2cjzfQPvnu"
      }
    },
    {
      "cell_type": "markdown",
      "source": [
        "####   a) Z-core method"
      ],
      "metadata": {
        "id": "NAWzmXy_QDjR"
      }
    },
    {
      "cell_type": "code",
      "source": [
        "from scipy.stats import zscore\n",
        "\n",
        "features = ['Quantity', 'UnitPrice']\n",
        "df_zscore = df.copy()\n",
        "for col in features:\n",
        "    df_zscore[f'{col}_zscore'] = zscore(df_zscore[col])\n",
        "\n",
        "# Threshold for Z-score\n",
        "z_thresh = 3\n",
        "df_zscore_filtered = df_zscore[(df_zscore['Quantity_zscore'].abs() < z_thresh) &\n",
        "                               (df_zscore['UnitPrice_zscore'].abs() < z_thresh)]"
      ],
      "metadata": {
        "id": "PlA0wjfDP02R"
      },
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "source": [
        "#### b)  IQR Method"
      ],
      "metadata": {
        "id": "nH2aPsg8NQzQ"
      }
    },
    {
      "cell_type": "code",
      "source": [
        "df_iqr = df.copy()\n",
        "for col in features:\n",
        "    Q1 = df_iqr[col].quantile(0.25)\n",
        "    Q3 = df_iqr[col].quantile(0.75)\n",
        "    IQR = Q3 - Q1\n",
        "    lower_bound = Q1 - 1.5 * IQR\n",
        "    upper_bound = Q3 + 1.5 * IQR\n",
        "    df_iqr = df_iqr[(df_iqr[col] >= lower_bound) & (df_iqr[col] <= upper_bound)]"
      ],
      "metadata": {
        "id": "JdffWgURMVxx"
      },
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "source": [
        "### 4.2 Visual Methods"
      ],
      "metadata": {
        "id": "PZEJ5ze-NvyY"
      }
    },
    {
      "cell_type": "markdown",
      "source": [
        "#### * Before Outlier Removal"
      ],
      "metadata": {
        "id": "0STK_BGWOiPD"
      }
    },
    {
      "cell_type": "code",
      "source": [
        "fig, axs = plt.subplots(2, 2, figsize=(14, 10))\n",
        "for i, col in enumerate(features):\n",
        "    sns.histplot(df[col], bins=30, kde=True, ax=axs[i][0], color='skyblue')\n",
        "    axs[i][0].set_title(f'Histogram BEFORE Outlier Removal: {col}')\n",
        "\n",
        "    sns.boxplot(x=df[col], ax=axs[i][1], color='salmon')\n",
        "    axs[i][1].set_title(f'Boxplot BEFORE Outlier Removal: {col}')\n",
        "plt.tight_layout()\n",
        "plt.show()"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 866
        },
        "id": "MsPhSaVrN6Oe",
        "outputId": "78a8e052-2226-481b-c53e-282e9e0cefc2"
      },
      "execution_count": null,
      "outputs": [
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 1400x1000 with 4 Axes>"
            ],
            "image/png": "...\n"
          },
          "metadata": {}
        }
      ]
    },
    {
      "cell_type": "markdown",
      "source": [
        "####  Z-score Filtering"
      ],
      "metadata": {
        "id": "7gIldQ46RXJZ"
      }
    },
    {
      "cell_type": "code",
      "source": [
        "df_zscore = df.copy()\n",
        "for col in features:\n",
        "    df_zscore[f'{col}_zscore'] = zscore(df_zscore[col])\n",
        "df_zscore_filtered = df_zscore[(df_zscore['Quantity_zscore'].abs() < 3) &\n",
        "                               (df_zscore['UnitPrice_zscore'].abs() < 3)]"
      ],
      "metadata": {
        "id": "2FGeW9VWRaFK"
      },
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "source": [
        "#### IQR Filtering"
      ],
      "metadata": {
        "id": "SGs9f1bIRd0x"
      }
    },
    {
      "cell_type": "code",
      "source": [
        "df_iqr = df.copy()\n",
        "for col in features:\n",
        "    Q1 = df_iqr[col].quantile(0.25)\n",
        "    Q3 = df_iqr[col].quantile(0.75)\n",
        "    IQR = Q3 - Q1\n",
        "    lower_bound = Q1 - 1.5 * IQR\n",
        "    upper_bound = Q3 + 1.5 * IQR\n",
        "    df_iqr = df_iqr[(df_iqr[col] >= lower_bound) & (df_iqr[col] <= upper_bound)]"
      ],
      "metadata": {
        "id": "yDLvfMz4Ri71"
      },
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "source": [
        "#### After Outlier Removal (Z-score)"
      ],
      "metadata": {
        "id": "OyaB60uxOg_8"
      }
    },
    {
      "cell_type": "code",
      "source": [
        "fig, axs = plt.subplots(2, 2, figsize=(14, 10))\n",
        "for i, col in enumerate(features):\n",
        "    sns.histplot(df_zscore_filtered[col], bins=30, kde=True, ax=axs[i][0], color='lightgreen')\n",
        "    axs[i][0].set_title(f'Histogram AFTER Z-score Removal: {col}')\n",
        "\n",
        "    sns.boxplot(x=df_zscore_filtered[col], ax=axs[i][1], color='lightblue')\n",
        "    axs[i][1].set_title(f'Boxplot AFTER Z-score Removal: {col}')\n",
        "plt.tight_layout()\n",
        "plt.show()"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 866
        },
        "id": "Tu_qFoohRxn8",
        "outputId": "eea47883-8615-4abc-9fd0-b201acb5cbbd"
      },
      "execution_count": null,
      "outputs": [
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 1400x1000 with 4 Axes>"
            ],
            "image/png": "...\n"
          },
          "metadata": {}
        }
      ]
   },
    {
      "cell_type": "markdown",
      "source": [
        "#### After Outlier Removal (IQR)"
      ],
      "metadata": {
        "id": "km8KYsqyR_Mh"
      }
    },
    {
      "cell_type": "code",
      "source": [
        "fig, axs = plt.subplots(2, 2, figsize=(14, 10))\n",
        "for i, col in enumerate(features):\n",
        "    sns.histplot(df_iqr[col], bins=30, kde=True, ax=axs[i][0], color='orange')\n",
        "    axs[i][0].set_title(f'Histogram AFTER IQR Removal: {col}')\n",
        "\n",
        "    sns.boxplot(x=df_iqr[col], ax=axs[i][1], color='purple')\n",
        "    axs[i][1].set_title(f'Boxplot AFTER IQR Removal: {col}')\n",
        "plt.tight_layout()\n",
        "plt.show()"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 866
        },
        "id": "YYP4a1GnR4vb",
        "outputId": "79c021af-fe4e-4f75-f3b4-0dab219ce47d"
      },
      "execution_count": null,
      "outputs": [
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 1400x1000 with 4 Axes>"
            ],
            "image/png": "...\n"
          },
          "metadata": {}
        }
      ]
    },
    {
      "cell_type": "markdown",
      "source": [
        "#### Summary"
      ],
      "metadata": {
        "id": "vZlhitTWShon"
      }
    },
    {
      "cell_type": "code",
      "source": [
        "print(\"Original dataset size:\", df.shape)\n",
        "print(\"After Z-score filtering:\", df_zscore_filtered.shape)\n",
        "print(\"After IQR filtering:\", df_iqr.shape)"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "g1mzyFA7SmrV",
        "outputId": "d734be39-d552-4be5-908f-6f89ce075f2e"
      },
      "execution_count": null,
           "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Original dataset size: (31150, 12)\n",
            "After Z-score filtering: (30843, 14)\n",
            "After IQR filtering: (26214, 12)\n"
          ]
        }
      ]
    },
    {
      "cell_type": "code",
      "source": [],
      "metadata": {
        "id": "-qta7RfhSnQB"
      },
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",